In [2]:
# Célula para instalar dependências
!pip install openai==0.28.1   
!pip install requests
!pip install flask
!pip install python-dotenv 


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [18]:
import os
import openai
import requests
import json
from flask import request, jsonify

# Usar os.getenv para ler as variáveis de ambiente
openai.api_type = os.getenv('OPENAI_API_TYPE', 'azure')
openai.api_version = os.getenv('OPENAI_API_VERSION', '2023-08-01-preview')
openai.api_base = os.getenv('OPENAI_API_BASE', 'https://project-name-uri.openai.azure.com/')
openai.api_key = os.getenv('OPENAI_API_KEY', 'sua-chave-de-api')
deployment_id = os.getenv('DEPLOYMENT_ID', 'deployment-id')

search_endpoint = os.getenv('SEARCH_ENDPOINT', 'https://az-search-endpoint-here.search.windows.net')
search_key = os.getenv('SEARCH_KEY', 'sua-chave-de-search-here')
search_index_name = os.getenv('SEARCH_INDEX_NAME', 'content')

In [19]:
# Célula 2: Função de Configuração BYOD para o OpenAI
def setup_byod(deployment_id):
    class BringYourOwnDataAdapter(requests.adapters.HTTPAdapter):
        def send(self, request, **kwargs):
            request.url = f"{openai.api_base}/openai/deployments/{deployment_id}/extensions/chat/completions?api-version={openai.api_version}"
            return super().send(request, **kwargs)

    session = requests.Session()
    session.mount(prefix=f"{openai.api_base}/openai/deployments/{deployment_id}", adapter=BringYourOwnDataAdapter())
    openai.requestssession = session

setup_byod(deployment_id)

In [22]:
# Célula 3: Simulação da Função 'chat' para Testes
def chat_simulator(message_text):
    try:
        completion = openai.ChatCompletion.create(
            messages=[{"role": "user", "content": message_text}],
            deployment_id=deployment_id,
            dataSources=[
                {
                    "type": "AzureCognitiveSearch",
                    "parameters": {
                        "endpoint": search_endpoint,
                        "key": search_key,
                        "indexName": search_index_name,
                    }
                }
            ]
        )
        decoded_text = completion.choices[0].message.content.encode('latin1').decode('unicode_escape')
        return decoded_text
    except Exception as e:
        return json.dumps({'error': str(e)}, indent=2)

In [23]:
# Célula 4: Executar a Simulação com uma Mensagem de Teste
test_message = "Como você está?"
print(chat_simulator(test_message))

Olá! Estou bem, obrigado por perguntar. Como posso ajudá-lo hoje? Lembre-se de que estou aqui para responder a perguntas relacionadas aos documentos recuperados. Se você tiver alguma dúvida, por favor, sinta-se à vontade para perguntar!
